In [1]:
import glob
import pandas as pd

from os.path import basename
import pickle
import json



# from level2.level2_roi_extraction import level2_roi_extractor
# from level2.level2_roi_extraction import load_rois, get_roi_data_for_beta
# from level2.level2_roi_extraction import get_roi_data_for_l2_betas, get_roi_data_for_multirun_l2_betas
# #import modules from files in a parallel directory "direct_regression"


# from direct_regression.get_all_series import get_beta_img, get_roi_data, get_moment_trial_type_revealed, get_behavioral_data_with_moment_trial_type_revealed, mask_3d_subject_image
# from direct_regression.get_all_series import get_all_subj_df
# from direct_regression.fmri_utils import *

import importlib


In [2]:
import modeling_utils
#importlib.reload(load_config)


In [3]:

config = modeling_utils.load_config("SST/direct_regression/config.yml")

glob_path = config['fmriprep_dir'] + config['nii_raw_path']

Benjamins-MacBook-Pro-2.local
{'dev_scripts_path': '/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/', 'level2_output_path': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/SST/', 'rural_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'rubric_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/DEV_scoring_rubrics', 'l2_analysis_files': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/', 'input_mat_path': '/Users/benjaminsmith/oregon/code/DEV_scripts/fMRI/fx/multiconds/SST/full_duration/conditions', 'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'dev_analysis_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/ana

## SST

### W1 W2 analysis

In [4]:
nonbids_data_path = config['nonbids_data_path']
dropbox_datapath = config['dropbox_data_dir']
data_quality = pd.read_excel(dropbox_datapath + "/DEV-BothSessionsDataQualityC_DATA.xlsx", engine = 'openpyxl')

In [5]:
sst_data_missing_includes = pd.read_csv(dropbox_datapath + "/post-processing-fmri-data-inclusion.csv")
sst_data_missing_includes['SST'].unique()


array([nan, 'Exclude'], dtype=object)

In [6]:
sst_data_missing_includes['SST Reason for exclusion'].unique()

array([nan, 'No T1w available, fMRIPrep could not be generated',
       'Exclude, no sst was collected for this subject',
       'behavioral data not available.',
       'No SST collected for this subject',
       'Motion error related to auto-motion-fmriprep. could not include.',
       'No T1 available, fMRIPrep could not be created'], dtype=object)

In [7]:
config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/"

'/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/'

In [8]:
#importlib.reload(level2.level2_utils)
#ultimately we can't make this completely task-agnostic, becasue we need to get a particular set of betas for each task
train_betas_with_data = modeling_utils.get_session_data_quality(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/",
    #nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    #ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    task='SST'
    #subj_wave_inclusion='all'
)



loaded 275 rows from data_by_ppt.csv
543 sessions remaining on the provisional useable_dev_id list from the redcap list after excluding subjects excluded by motion quality process.
(250, 3) (275, 173) (287, 176)
(543, 41) (571, 43) (572, 86)
(287, 177) (572, 86) (574, 262)
['beta_subject_id', 'SID', 'subject_id', 'redcap_dev_id', 'redcap_wave', 'motion_exclude_wave', 'motion_exclude_missing wave', 'wave_id']


In [9]:
train_betas_with_data.to_csv(dropbox_datapath + "/beta_file_matches_sst.csv")

## Case study: SST data

Let's try to explain the size of the SST dataset.

In [10]:
SST_selected_data = (
    train_betas_with_data[
        #redcap
        (train_betas_with_data.redcap_SST=='No reported problems')
        #motion exclude
        & (train_betas_with_data.motion_exclude_SST_Exclude.isna()==True)
        & (train_betas_with_data.motion_exclude_SST_Exclude != '')
         #betas
        & (train_betas_with_data.spm_l2_path.isna()==False)
        & (train_betas_with_data.spm_l2_path != '')
         #behavioral
        & (train_betas_with_data.data_by_ppt_merge_status=='participant_present')
    ]
)

In [11]:
#now we separate by wave
SST_selected_data_w1 = SST_selected_data[SST_selected_data.wave_id==1]
#we have separate wave filters that also need to be applied.
SST_selected_data_w1.motion_exclude_SST_Exclude
SST_selected_data_w2 = SST_selected_data[SST_selected_data.wave_id==2]

In [12]:
#now I guess we check to see which subjects exist in wave 1, wave 2, or both waves!
w1_subj_ids = SST_selected_data_w1.subject_id
w2_subj_ids = SST_selected_data_w2.subject_id
#now get the count of each, then the count of the intersection
w1_subj_ids.shape
w2_subj_ids.shape
#count of intersection
len(set(w1_subj_ids) & set(w2_subj_ids))

178

This may be a few more than we have had in the pipeline because I haven't used the now-obsolete exclusion list.

Now let's try again, getting that data, with a new function that will whittle down to just the subjects who have runs in each wave

In [13]:

importlib.reload(modeling_utils)
#importlib.reload(level2.level2_utils)
#ultimately we can't make this completely task-agnostic, becasue we need to get a particular set of betas for each task
subjects_in_all_waves = modeling_utils.get_sst_data_for_confirmed_sessions_across_tasks(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/",
    #nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    #ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    subj_wave_inclusion='all'
)



loaded 275 rows from data_by_ppt.csv


543 sessions remaining on the provisional useable_dev_id list from the redcap list after excluding subjects excluded by motion quality process.
(250, 3) (275, 173) (287, 176)
(543, 41) (571, 43) (572, 86)
(287, 177) (572, 86) (574, 262)
['beta_subject_id', 'SID', 'subject_id', 'redcap_dev_id', 'redcap_wave', 'motion_exclude_wave', 'motion_exclude_missing wave', 'wave_id']
subjects who did or did not pass the scanner room report check:
redcap_SST
Investigate               6
May not be usable         5
Minor problems           31
No reported problems    480
Unusable                 20
dtype: int64
True     480
False     94
Name: redcap_SST, dtype: int64
subjects who did or did not pass the motion check:
True     545
False     29
Name: motion_exclude_SST_Exclude, dtype: int64
subjects who did or did not have beta data:
True     501
False     73
Name: spm_l2_path, dtype: int64
subjects who did or did not have behavioral data:
True     534
False     40
Name: data_by_ppt_merge_status, dtype:

In [14]:
subjects_in_all_waves

,beta_subject_id,SID,subject_id,redcap_dev_id,redcap_wave,motion_exclude_wave,motion_exclude_missing wave,wave_id,data_by_ppt_merge_status,redcap_SST,...,motion_exclude_WTP1 motion,motion_exclude_WTP2 SDC,motion_exclude_WTP2 reg,motion_exclude_WTP2 motion,motion_exclude_WTP3 SDC,motion_exclude_WTP3 reg,motion_exclude_WTP3 motion,motion_exclude_WTP4 SDC,motion_exclude_WTP4 reg,motion_exclude_WTP4 motion
23,DEV014,DEV014,DEV014,DEV014,1.0,1.0,NaN,1.0,participant_present,No reported problems,...,0,x,x,0,x,x,0,x,x,0
24,DEV014,DEV014,DEV014,DEV014,2.0,2.0,NaN,2.0,participant_present,No reported problems,...,0,x,x,0,x,x,0,x,x,NaN
25,DEV015,DEV015,DEV015,DEV015,1.0,1.0,NaN,1.0,participant_present,No reported problems,...,0,x,x,0,x,x,0,x,x,0
26,DEV015,DEV015,DEV015,DEV015,2.0,2.0,NaN,2.0,participant_present,No reported problems,...,1,x,x,0,x,x,0,x,x,NaN
29,DEV017,DEV017,DEV017,DEV017,1.0,1.0,NaN,1.0,participant_present,No reported problems,...,0,x,x,0,x,x,0,x,x,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,DEV309,DEV309,DEV309,DEV309,2.0,2.0,NaN,2.0,participant_present,No reported problems,...,NaN,bad ghosting,x,NaN,x,x,NaN,x,x,NaN
565,DEV310,DEV310,DEV310,DEV310,1.0,1.0,NaN,1.0,participant_present,No reported problems,...,NaN,ghosting with slight overlap,ghosting with slight overlap,NaN,ghosting with slight overlap,ghosting with slight overlap,NaN,ghosting with slight overlap,ghosting with slight overlap,NaN
566,DEV310,DEV310,DEV310,DEV310,2.0,2.0,NaN,2.0,participant_present,No reported problems,...,NaN,"ghosting, no overlap","ghosting, no overlap",NaN,"ghosting, no overlap","ghosting, no overlap",NaN,"ghosting, no overlap","ghosting, no overlap",NaN
569,DEV312,DEV312,DEV312,DEV312,1.0,1.0,NaN,1.0,participant_present,No reported problems,...,x,x,x,x,x,x,x,x,x,x


# WTP

In [141]:
importlib.reload(modeling_utils)
#importlib.reload(level2.level2_utils)
#ultimately we can't make this completely task-agnostic, becasue we need to get a particular set of betas for each task
all_sessions_wtp = modeling_utils.get_task_data_for_confirmed_sessions(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/",
    #nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    #ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    task='WTP'
)



loaded 275 rows from data_by_ppt.csv
543 sessions remaining on the provisional useable_dev_id list from the redcap list after excluding subjects excluded by motion quality process.
(250, 3) (275, 173) (287, 176)
(543, 41) (570, 43) (571, 86)
(287, 177) (571, 86) (573, 262)
['beta_subject_id', 'SID', 'subject_id', 'redcap_dev_id', 'redcap_wave', 'motion_exclude_wave', 'motion_exclude_missing wave', 'wave_id']
subjects who did or did not pass the scanner room report check:
True     505
False     68
Name: redcap_WTP_quality_any, dtype: int64
subjects who did or did not pass the motion check:
True     547
False     26
Name: motion_exclude_WTP_Exclude, dtype: int64
subjects who did or did not have beta data:
True     500
False     73
Name: spm_l2_path, dtype: int64
subjects who did or did not have behavioral data:
True     533
False     40
Name: data_by_ppt_merge_status, dtype: int64


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:487: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_by_session[col + '_quality'] = all_data_by_session[col].apply(lambda x: 1 if x=='No reported problems' else 0)
/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:487: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_by_session[col + '_quality'] = all_data_by_session[col].apply(lambda x: 1 if x=='No reported proble

In [142]:

importlib.reload(modeling_utils)
#get columns matching regex 'redcap_WTP\d_quality'
#and then create a list of the columns, then jsonify the list
subjects_in_all_waves = modeling_utils.get_subject_wise_table_with_task_counts(all_sessions_wtp)

In [149]:
subjects_in_all_waves.WTP_quality_list

array(['[[1, 2, 3, 4], [1, 2, 3, 4]]', '[[1, 2, 4], [1, 2, 3, 4]]',
       '[[1, 3, 4], [1, 2, 3, 4]]', '[[2, 3, 4], [1, 2, 3, 4]]',
       '[[3, 4], [1, 2, 3, 4]]', '[[1, 2, 3, 4], [3, 4]]',
       '[[3, 4], [2, 3, 4]]', '[[4], [1, 2, 3, 4]]',
       '[[1, 3, 4], [2, 3, 4]]', '[[1, 2, 3, 4], [1, 2, 4]]',
       '[[2, 3, 4], [2, 3, 4]]'], dtype=object)

In [147]:
#now save a tab delimited file with a list of subject IDs and json codes
subjects_in_all_waves.loc[:,['SID','WTP_quality_list']].to_csv(dropbox_datapath + "/wtp_run_data_quality.csv", index=False,header=False, sep='\t')
